In [74]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
Top4 = ['India' , 'Pakistan' , 'Australia' , 'South Africa']
batsmen = pd.read_csv('batting_scrapped.csv')
bowler_df = pd.read_csv('bowlers_scrapped.csv')
print(batsmen)

                Player    Country              Opponent  Matches  Innings  \
0          Virat Kohli      India           Afghanistan        3        2   
1          Virat Kohli      India             Australia       48       46   
2          Virat Kohli      India            Bangladesh       16       16   
3          Virat Kohli      India               England       36       36   
4          Virat Kohli      India               Ireland        2        2   
...                ...        ...                   ...      ...      ...   
1119  Charith Asalanka  Sri Lanka          South Africa        4        4   
1120  Charith Asalanka  Sri Lanka  United Arab Emirates        1        1   
1121  Charith Asalanka  Sri Lanka           West Indies        1        0   
1122  Charith Asalanka  Sri Lanka              Zimbabwe        4        3   
1123  Charith Asalanka  Sri Lanka          Overall (15)       50       44   

      Not_Out  Hundreds  Fifties  Runs  Average  
0           1         0  

We will use scrapped data of task1 in team prediction.

We will only consider potential perfomance of players against teams in Top 4

In [75]:
# filtering player stats for Top 4 teams
batsmen= batsmen[batsmen['Opponent'].isin(Top4)]
batsmen= batsmen[batsmen['Country'].isin(Top4)]
bowler_df= bowler_df[bowler_df['Opponent'].isin(Top4)]
bowler_df= bowler_df[bowler_df['Country'].isin(Top4)]
print(batsmen)

             Player   Country      Opponent  Matches  Innings  Not_Out  \
1       Virat Kohli     India     Australia       48       46        3   
8       Virat Kohli     India      Pakistan       16       16        3   
9       Virat Kohli     India  South Africa       31       29        6   
16     Rohit Sharma     India     Australia       44       44        4   
24     Rohit Sharma     India      Pakistan       19       19        2   
..              ...       ...           ...      ...      ...      ...   
472     Shadab Khan  Pakistan         India        8        6        2   
476     Shadab Khan  Pakistan  South Africa       10        7        2   
482  Mohammad Nawaz  Pakistan     Australia        2        2        0   
485  Mohammad Nawaz  Pakistan         India        3        2        1   
490  Mohammad Nawaz  Pakistan  South Africa        2        2        0   

     Hundreds  Fifties  Runs  Average  
1           8       13  2313    53.79  
8           3        2   678   

In [63]:

print(bowler_df)

             Player       Country      Opponent  Matches  Overs  Maidens  \
297  Shardul Thakur         India     Australia        5   31.0        1   
303  Shardul Thakur         India      Pakistan        3    6.0        0   
304  Shardul Thakur         India  South Africa        6   38.5        2   
310  Jasprit Bumrah         India     Australia       20  185.3        9   
315  Jasprit Bumrah         India      Pakistan        8   51.1        5   
316  Jasprit Bumrah         India  South Africa       12   95.0        3   
322   Kuldeep Yadav         India     Australia       21  188.0        4   
329   Kuldeep Yadav         India      Pakistan        6   45.0        1   
330   Kuldeep Yadav         India  South Africa       11   87.2        3   
336  Mohammad Shami         India     Australia       24  198.4        8   
343  Mohammad Shami         India      Pakistan        3   28.0        5   
344  Mohammad Shami         India  South Africa        5   40.0        2   
350  Mohamma

We have to convert string data to numeric type

In [76]:
batsmen['Average'] = pd.to_numeric(batsmen['Average'], errors='coerce')
batsmen['Matches'] = pd.to_numeric(batsmen['Matches'], errors='coerce')
batsmen['Innings'] = pd.to_numeric(batsmen['Innings'], errors='coerce')
batsmen['Hundreds'] = pd.to_numeric(batsmen['Hundreds'], errors='coerce')
batsmen['Fifties'] = pd.to_numeric(batsmen['Fifties'], errors='coerce')
batsmen['Runs'] = pd.to_numeric(batsmen['Runs'], errors='coerce')

We are assuming that Playing 11 will consist of 6 batsmen and 5 bowlers.
We will select Top performers for each team

We have to encode player and country names.

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder_players = LabelEncoder()
batsmen['Encoded_Player'] = label_encoder_players.fit_transform(batsmen['Player'])
label_encoder_teams = LabelEncoder()
label_encoder_teams.fit(Top4)
batsmen['Encoded_Country'] = label_encoder_teams.fit_transform(batsmen['Country'])
batsmen['Encoded_Opponent'] = label_encoder_teams.fit_transform(batsmen['Opponent'])  

In [10]:
Train_df = batsmen[['Encoded_Player' , 'Encoded_Country' , 'Encoded_Opponent' , 'Matches' , 'Runs']]
print(Train_df)

     Encoded_Player  Encoded_Country  Encoded_Opponent  Matches  Runs
1                32                1                 0       48  2313
8                32                1                 2       16   678
9                32                1                 3       31  1504
16               22                1                 0       44  2332
24               22                1                 2       19   873
..              ...              ...               ...      ...   ...
472              25                2                 1        8    60
476              25                2                 3       10   145
482              17                2                 0        2    15
485              17                2                 1        3    19
490              17                2                 3        2    28

[95 rows x 5 columns]


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [12]:
X = Train_df[['Encoded_Player' , 'Encoded_Country' , 'Encoded_Opponent' , 'Matches']]
y = Train_df[['Runs']]
X['Matches'] = pd.to_numeric(X['Matches'], errors='coerce')
y['Runs'] = pd.to_numeric(y['Runs'], errors='coerce')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)
y_pred_dnn = model.predict(X_test)
r2_test = r2_score(y_test, y_pred_dnn)
print(f'R-squared on Test Set: {r2_test}')

1/1 [==============================] - 0s 37ms/step
R-squared on Test Set: 0.8244753683024628


In [15]:
player_country = batsmen[['Player' , 'Encoded_Player' , 'Country' , 'Encoded_Country']]
player_country.drop_duplicates(inplace=True)
print(player_country)

                    Player  Encoded_Player       Country  Encoded_Country
1              Virat Kohli              32         India                1
16            Rohit Sharma              22         India                1
31            Shubman Gill              27         India                1
44            Shreyas Iyer              26         India                1
55        Suryakumar Yadav              29         India                1
65           Hardik Pandya              10         India                1
76         Ravindra Jadeja              20         India                1
91     Ravichandran Ashwin              19         India                1
104           Temba Bavuma              30  South Africa                3
115        Reeza Hendricks              21  South Africa                3
125          Aiden Markram               2  South Africa                3
138           David Miller               6  South Africa                3
152  Rassie van der Dussen            

In [16]:
# A helper function to get the encoded_country

def get_encoded_country(input_country):
    
    for index, row in player_country.iterrows():
        if(row['Country']==input_country) :
            return row['Encoded_Country']
    return 'Error'

Now We will calculate sum of predicted runs against other three teams.

In [18]:
player_runs = {}
for index , row in player_country.iterrows() :
    player_runs[row['Player']] = 0

cnt = 0
for opponent in Top4 :
    for index , row in player_country.iterrows() :
        if(row['Country']!=opponent) :  
            
            runs = model.predict([[row['Encoded_Player'], row['Encoded_Country'], get_encoded_country(opponent), 1]])
            player_runs[row['Player']]+= (int(runs[0][0])) 

1/1 [==============================] - 0s 17ms/step


In [32]:
def get_player_country(Name) :
    for index , row in player_country.iterrows() :
        if(row['Player'] == Name):
            return row['Country']
    return 'None'



from collections import OrderedDict
sorted_list = OrderedDict(sorted(player_runs.items(), key=lambda item: item[1], reverse=True))

# Getting Top Batsmen of India

print('---------------  Indian Batsmen Potential Scores ------------------')
print("")
for pl in sorted_list :
    if(get_player_country(pl)=='India'):
        print(f'{pl} : {sorted_list[pl]}')

---------------  Indian Batsmen Potential Scores ------------------

Virat Kohli : 326
Suryakumar Yadav : 299
Shubman Gill : 280
Shreyas Iyer : 271
Rohit Sharma : 233
Ravindra Jadeja : 215
Ravichandran Ashwin : 205
Hardik Pandya : 126


In [34]:
# Getting Top Batsmen of Pakistan

print('---------------  Pakistan Batsmen Potential Scores ------------------')
print("")
for pl in sorted_list :
    if(get_player_country(pl)=='Pakistan'):
        print(f'{pl} : {sorted_list[pl]}')

---------------  Pakistan Batsmen Potential Scores ------------------

Shadab Khan : 248
Saud Shakeel : 229
Mohammad Nawaz : 175
Imam-ul-Haq : 132
Iftikhar Ahmed : 125
Fakhar Zaman : 103
Babar Azam : 86
Abdullah Shafique : 84
Agha Salman : 83


In [35]:
# Getting Top Batsmen of Pakistan

print('---------------  Australian Batsmen Potential Scores ------------------')
print("")
for pl in sorted_list :
    if(get_player_country(pl)=='Australia'):
        print(f'{pl} : {sorted_list[pl]}')

---------------  Australian Batsmen Potential Scores ------------------

Travis Head : 333
Steven Smith : 304
Sean Abbot : 267
Mitchel Marsh : 192
Marnus Labuschagne : 182
Marcus Stoinis : 173
Glenn Maxwell : 128
David Warner : 112
Cameroon Green : 99


In [36]:
# Getting Top Batsmen of Sauth Africa

print('---------------  South African Batsmen Potential Scores ------------------')
print("")
for pl in sorted_list :
    if(get_player_country(pl)=='South Africa'):
        print(f'{pl} : {sorted_list[pl]}')

---------------  South African Batsmen Potential Scores ------------------

Temba Bavuma : 284
Reeza Hendricks : 200
Rassie van der Dussen : 175
Marco Jansen : 135
David Miller : 94
Aiden Markram : 91
Andile Phehlukwayo : 88


We will do similar predictions for bowlers.

In [77]:
bowler_df['Matches'] = pd.to_numeric(bowler_df['Matches'] , errors = 'coerce')
bowler_df['Overs'] = pd.to_numeric(bowler_df['Overs'] , errors = 'coerce')
bowler_df['Runs'] = pd.to_numeric(bowler_df['Runs'] , errors = 'coerce')
bowler_df['Maidens'] = pd.to_numeric(bowler_df['Maidens'] , errors = 'coerce')
bowler_df['Wickets'] = pd.to_numeric(bowler_df['Wickets'] , errors = 'coerce')

In [78]:
label_encoder_bowlers = LabelEncoder()
bowler_df['Encoded_Player'] = label_encoder_players.fit_transform(bowler_df['Player'])
print(bowler_df)

             Player       Country      Opponent  Matches  Overs  Maidens  \
297  Shardul Thakur         India     Australia        5   31.0        1   
303  Shardul Thakur         India      Pakistan        3    6.0        0   
304  Shardul Thakur         India  South Africa        6   38.5        2   
310  Jasprit Bumrah         India     Australia       20  185.3        9   
315  Jasprit Bumrah         India      Pakistan        8   51.1        5   
316  Jasprit Bumrah         India  South Africa       12   95.0        3   
322   Kuldeep Yadav         India     Australia       21  188.0        4   
329   Kuldeep Yadav         India      Pakistan        6   45.0        1   
330   Kuldeep Yadav         India  South Africa       11   87.2        3   
336  Mohammad Shami         India     Australia       24  198.4        8   
343  Mohammad Shami         India      Pakistan        3   28.0        5   
344  Mohammad Shami         India  South Africa        5   40.0        2   
350  Mohamma

In [79]:
bowler_df['Encoded_Country'] = label_encoder_teams.fit_transform(bowler_df['Country'])
bowler_df['Encoded_Opponent'] = label_encoder_teams.fit_transform(bowler_df['Opponent'])

print(bowler_df)

             Player       Country      Opponent  Matches  Overs  Maidens  \
297  Shardul Thakur         India     Australia        5   31.0        1   
303  Shardul Thakur         India      Pakistan        3    6.0        0   
304  Shardul Thakur         India  South Africa        6   38.5        2   
310  Jasprit Bumrah         India     Australia       20  185.3        9   
315  Jasprit Bumrah         India      Pakistan        8   51.1        5   
316  Jasprit Bumrah         India  South Africa       12   95.0        3   
322   Kuldeep Yadav         India     Australia       21  188.0        4   
329   Kuldeep Yadav         India      Pakistan        6   45.0        1   
330   Kuldeep Yadav         India  South Africa       11   87.2        3   
336  Mohammad Shami         India     Australia       24  198.4        8   
343  Mohammad Shami         India      Pakistan        3   28.0        5   
344  Mohammad Shami         India  South Africa        5   40.0        2   
350  Mohamma

In [80]:
Bowler_Train_df = bowler_df[['Encoded_Player' , 'Encoded_Country' , 'Encoded_Opponent' , 'Matches' , 'Wickets']]
print(Bowler_Train_df)

     Encoded_Player  Encoded_Country  Encoded_Opponent  Matches  Wickets
297              15                1                 0        5        3
303              15                1                 2        3        1
304              15                1                 3        6        8
310               3                1                 0       20       28
315               3                1                 2        8        7
316               3                1                 3       12       17
322               7                1                 0       21       31
329               7                1                 2        6       12
330               7                1                 3       11       26
336              11                1                 0       24       38
343              11                1                 2        3        5
344              11                1                 3        5       13
350              12                1               

In [81]:
X = Bowler_Train_df[['Encoded_Player' , 'Encoded_Country' , 'Encoded_Opponent' , 'Matches']]
y = Bowler_Train_df[['Wickets']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [82]:
model2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model2.compile(optimizer='adam', loss='mean_squared_error')
model2.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)
y_pred_dnn = model2.predict(X_test)
r2_test = r2_score(y_test, y_pred_dnn)
print(f'R-squared on Test Set: {r2_test}')

1/1 [==============================] - 0s 31ms/step
R-squared on Test Set: 0.838801857131726


In [83]:
bowler_country = bowler_df[['Player' , 'Encoded_Player' , 'Country' , 'Encoded_Country']]
bowler_country.drop_duplicates(inplace=True)
print(bowler_country)

             Player  Encoded_Player       Country  Encoded_Country
297  Shardul Thakur              15         India                1
310  Jasprit Bumrah               3         India                1
322   Kuldeep Yadav               7         India                1
336  Mohammad Shami              11         India                1
350  Mohammad Siraj              12         India                1
363  Keshav Maharaj               6  South Africa                3
374     Luigi Ngidi               8  South Africa                3
387   Kagiso Rabada               5  South Africa                3
399  Tabraiz Shamsi              16  South Africa                3
414     Pat Cummins              13     Australia                0
427      Adam Zampa               0     Australia                0
440  Mitchell Starc              10     Australia                0
454  Josh Hazlewood               4     Australia                0
528      Haris Rauf               1      Pakistan             

In [84]:
# A helper function to get the encoded_country

def get_bowler_country(Name):
    
    for index, row in bowler_country.iterrows():
        if(row['Player']==Name) :
            return row['Country']
    return 'Error'

In [85]:
player_wickets = {}
for index , row in bowler_country.iterrows() :
    player_wickets[row['Player']] = 0

for opponent in Top4 :
    for index , row in bowler_country.iterrows() :
        if(row['Country']!=opponent) :  
            
            wickets = model2.predict([[row['Encoded_Player'], row['Encoded_Country'], get_encoded_country(opponent), 1]])
            player_wickets[row['Player']]+= (int(wickets[0][0]))    

1/1 [==============================] - 0s 21ms/step


In [90]:
sorted_bowlers = OrderedDict(sorted(player_wickets.items(), key=lambda item: item[1], reverse=True))

# Getting top wicket takers of India


print('-------------Top Potential Wicket takers of India-------------')
print('Player : Potential Wickets in playoffs')

for pl in sorted_bowlers :
    if(get_bowler_country(pl)=='India'):
        print(f'{pl} : {sorted_bowlers[pl]}')

-------------Top Potential Wicket takers of India-------------
Player : Potential Wickets in playoffs
Shardul Thakur : 6
Mohammad Siraj : 6
Mohammad Shami : 5
Kuldeep Yadav : 4
Jasprit Bumrah : 3


In [91]:
# Getting top wicket takers of Pakistan


print('-------------Top Potential Wicket takers of Pakistan-------------')
print('Player : Potential Wickets in playoffs')

for pl in sorted_bowlers :
    if(get_bowler_country(pl)=='Pakistan'):
        print(f'{pl} : {sorted_bowlers[pl]}')

-------------Top Potential Wicket takers of Pakistan-------------
Player : Potential Wickets in playoffs
Usama Mir : 7
Shaheen Afridi : 6
Md. Wasim Jr. : 5
Haris Rauf : 4
Hasan Ali : 4


In [92]:

print('-------------Top Potential Wicket takers of Austalia-------------')
print('Player : Potential Wickets in playoffs')

for pl in sorted_bowlers :
    if(get_bowler_country(pl)=='Australia'):
        print(f'{pl} : {sorted_bowlers[pl]}')

-------------Top Potential Wicket takers of Austalia-------------
Player : Potential Wickets in playoffs
Pat Cummins : 6
Mitchell Starc : 5
Adam Zampa : 3
Josh Hazlewood : 3


In [93]:

print('------------- Top Potential Wicket takers of South Africa -------------')
print('Player : Potential Wickets in playoffs')

for pl in sorted_bowlers :
    if(get_bowler_country(pl)=='South Africa'):
        print(f'{pl} : {sorted_bowlers[pl]}')

------------- Top Potential Wicket takers of South Africa -------------
Player : Potential Wickets in playoffs
Tabraiz Shamsi : 7
Keshav Maharaj : 5
Luigi Ngidi : 5
Kagiso Rabada : 5


Here we have predicted best peforming bowlers and batsmen of All the Top 4 teams. Playing 11 will be selected according to their predicted peformance against other three countries. We know that we also have to consider wicket keepers and all rounders but the request limit is reached for data scrapping that's why we could not scrap data of wicket-keepers and all rounders.